## Dataset del Curso

Este curso usa un modelo dimensional simplificado para e-commerce:

**Dimensiones:**
- `dim_clientes` (20 registros): cliente_id, nombre, email, ciudad, segmento, region_id, fecha_alta
- `dim_productos` (10 registros): producto_id, nombre, categoria, precio_unitario, costo_unitario, activo
- `dim_regiones` (3 registros): region_id, nombre_region, pais

**Hechos:**
- `fact_ventas` (~200 registros): venta_id, cliente_id, producto_id, region_id, fecha, cantidad, descuento_pct, canal
- `fact_suscripciones` (~50 registros): suscripcion_id, cliente_id, fecha_inicio, fecha_fin, monto_mensual, estado
- `fact_inventario` (~30 registros): inventario_id, producto_id, fecha, cantidad

**Prerequisito:** ejecutar `dataset_setup.sql` para crear y poblar estas tablas.

## 🟢 Ejercicio Básico
Sin ejecutar consultas aún, dibuja en papel cómo se relacionan:
- Un cliente con sus ventas (¿qué columna conecta ambas tablas?)
- Una venta con el producto vendido (¿FK?)
- Una región con sus clientes

## 🟠 Ejercicio Intermedio
Responde conceptualmente:
1. ¿Puedo insertar una venta con `cliente_id = 999` si ese cliente no existe? ¿Por qué?
2. ¿Qué pasa si elimino un cliente que tiene 10 ventas registradas? (hint: integridad referencial)
3. ¿Por qué `precio_unitario` está en `dim_productos` y no en `fact_ventas`?

## 🔴 Reto Conceptual
Diseña en papel una nueva tabla `fact_devoluciones` que registre:
- Qué producto se devolvió
- Qué cliente lo devolvió
- Fecha de devolución
- Motivo (texto)

Define:
- PK (¿qué columna?)
- FKs (¿a qué tablas apunta?)
- Columnas adicionales necesarias

---

## Errores Comunes al Empezar

❌ **Pensar que SQL es programación**: SQL es declarativo ("qué quiero") no imperativo ("cómo hacerlo paso a paso")
❌ **No entender PKs**: "¿por qué necesito un ID si ya tengo el nombre?" → nombres se repiten, IDs no
❌ **Duplicar información**: guardar nombre_cliente en fact_ventas → si cambia el nombre, inconsistencia
❌ **No respetar FKs**: insertar IDs inventados rompe relaciones
❌ **Confundir tabla con archivo Excel**: Excel es flexible pero frágil; BD relacional garantiza consistencia

---

## Próximos Pasos
Ahora que entiendes el modelo relacional:
- **Siguiente:** `02_select_basico.ipynb` → tus primeras consultas SELECT
- **Recurso:** Lee `dataset_setup.sql` para ver el DDL (CREATE TABLE) completo

In [ ]:
-- Ver estructura de una tabla (columnas, tipos de datos)
-- En SQL Server Management Studio o Azure Data Studio:
-- Click derecho en tabla → "Script Table as CREATE"

-- Alternativa: consultar metadata del sistema
SELECT 
    c.name AS columna,
    t.name AS tipo_dato,
    c.max_length AS longitud,
    c.is_nullable AS permite_null
FROM sys.columns c
INNER JOIN sys.types t ON c.user_type_id = t.user_type_id
WHERE c.object_id = OBJECT_ID('dbo.dim_clientes')
ORDER BY c.column_id;

-- Ver claves primarias y foráneas
EXEC sp_help 'dbo.fact_ventas';

/*
💡 Interpretación:
- PK (Primary Key): usualmente la primera columna (cliente_id, producto_id)
- FK (Foreign Key): columnas que terminan en _id y referencian otra tabla
- Tipos comunes: INT (enteros), VARCHAR (texto), DATE (fechas), DECIMAL (números con decimales)
*/

## Tipos de Tablas en Modelos Dimensionales

### **Tablas de Dimensiones (dim_*)**
Contienen atributos descriptivos (quién, qué, dónde, cuándo):
- `dim_clientes`: quién compra (nombre, ciudad, segmento)
- `dim_productos`: qué se vende (nombre, categoría, precio)
- `dim_regiones`: dónde ocurre (país, región, zona)
- `dim_tiempo`: cuándo (fecha, mes, trimestre, año)

**Características:**
- Pocas filas (100s a 10,000s)
- Cambian raramente (nombre de producto se actualiza ocasionalmente)
- Se relacionan con hechos mediante FKs

### **Tablas de Hechos (fact_*)**
Contienen métricas numéricas y FKs a dimensiones:
- `fact_ventas`: registra cada transacción (cantidad, descuento, FK a cliente, producto, región, fecha)
- `fact_inventario`: snapshot de stock (cantidad disponible, FK a producto, fecha)
- `fact_suscripciones`: estado de suscripciones (FK a cliente, fecha inicio/fin)

**Características:**
- Muchas filas (millones)
- Crecen constantemente (nueva venta = nueva fila)
- Contienen FKs a todas las dimensiones relevantes

### 1️⃣ **Tabla (Table)**
Colección de datos organizados en filas y columnas:
- **Fila (Row/Registro)**: representa una entidad (ej. un cliente)
- **Columna (Column/Campo)**: representa un atributo (ej. nombre del cliente)

```
Tabla: dim_clientes
┌─────────────┬─────────────┬──────────────┬─────────┐
│ cliente_id  │ nombre      │ email        │ ciudad  │
├─────────────┼─────────────┼──────────────┼─────────┤
│ 1           │ Ana García  │ ana@mail.com │ Lima    │
│ 2           │ Luis Torres │ luis@mail.mx │ CDMX    │
│ 3           │ María Ruiz  │ maria@mail.es│ Madrid  │
└─────────────┴─────────────┴──────────────┴─────────┘
```

### 2️⃣ **Clave Primaria (Primary Key - PK)**
Columna que identifica **únicamente** cada fila:
- `cliente_id` = 1 solo existe UNA vez
- No puede ser NULL (debe tener valor)
- No puede duplicarse

¿Por qué? Para poder referenciar "el cliente 1" sin ambigüedad.

### 3️⃣ **Clave Foránea (Foreign Key - FK)**
Columna que **referencia** la PK de otra tabla:

```
Tabla: fact_ventas
┌──────────┬─────────────┬──────────────┬──────────┐
│ venta_id │ cliente_id  │ producto_id  │ cantidad │
├──────────┼─────────────┼──────────────┼──────────┤
│ 101      │ 1           │ 201          │ 5        │
│ 102      │ 2           │ 202          │ 10       │
└──────────┴─────────────┴──────────────┴──────────┘
         FK ──────────▶ dim_clientes(cliente_id)
```

**Integridad referencial**: no puedes insertar `cliente_id = 999` si ese cliente no existe en `dim_clientes`.

# 1.1 Introducción al Modelo Relacional

## 🎯 ¿Para qué existe SQL?
SQL (Structured Query Language) es el lenguaje universal para comunicarte con **bases de datos relacionales**. Te permite:
- **Consultar** datos (SELECT)
- **Insertar** nuevos registros (INSERT)
- **Actualizar** información existente (UPDATE)
- **Eliminar** datos (DELETE)
- **Definir** estructura de tablas (CREATE TABLE)

## 📚 ¿Por qué "relacional"?

Las bases de datos relacionales organizan información en **tablas** (como hojas de Excel) que se **relacionan** entre sí mediante claves:

### Ejemplo del mundo real:
Imagina una librería:
- **Tabla Libros**: libro_id, título, autor_id, precio
- **Tabla Autores**: autor_id, nombre, país
- **Relación**: cada libro está conectado a un autor mediante `autor_id`

Sin modelo relacional, tendrías que duplicar el nombre del autor en cada libro → si cambia el nombre, actualizas en 100 lugares ❌

Con modelo relacional: autor se guarda UNA vez, los libros solo referencian su ID → cambio en 1 lugar ✅

## 🔧 Conceptos Fundamentales

## Referencias a dataset
Tablas disponibles ya creadas por dataset_setup.sql: dim_clientes, dim_productos, dim_regiones, fact_ventas, fact_suscripciones, fact_inventario.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[Siguiente ➡️](02_select_basico.ipynb)
---
